<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 371 Homework 8

Partners: Brian Janger, Matthew Wang, Caleb Watson

### Problem 0 (3 points)

## Part 1: Kernels 

### Problem 1.1 (Exam Style)

First step, we put the conditions in matrix form: 

$
\begin{bmatrix}
K(x_1,x_1) & K(x_1,x_2)\\
K(x_2,x_1) & K(x_2,x_2)
\end{bmatrix}
=
\begin{bmatrix}
1 & 2\\
2 & 1
\end{bmatrix}
$

To check that it is a valid Kernel, we test if the matrix is positive semi-definite. We find the eigenvalues of the symmetrical matrix. 

The eigenvalues of the matrix are $\lambda_1=-1, \lambda_2=3$. Since one of the eigenvalues is negative, the matrix is not positive semi-definite. Therefore, no function $\rho(x)$ exists.

### Problem 1.2 (Exam Style)

The function $K(x,\xi)=(x^T\xi+c)^2$ in $R^3$ is $K(x,\xi)=(x^T\xi+c)^2=(x_1\xi_1 + x_2\xi_2 + x_3\xi_3 + c)^2$

Expanding the function out, we get 

$x_1^2\xi_1^2 + x_2^2\xi_2^2 + x_3^2\xi_3^2 + 2x_1x_2\xi_1\xi_2 + 2x_1x_3\xi_1\xi_3 + 2x_2x_3\xi_2\xi_3 + 2x_1\xi_1c + 2x_2\xi_2c + 2x_3\xi_3c + c^2$

Therefore 

$\rho(x)=\rho(x_1,x_2,x_3)=(c,x_1,x_2,x_3,\sqrt{2}x_1x_2,\sqrt{2}x_1x_3,\sqrt{2}x_2x_3,\sqrt{2}x_1,\sqrt{2}x_2,\sqrt{2}x_3)^T$

There are 10 monomials, so $e=10$

### Problem 1.3 (Exam Style)

$e={d+k \choose k}$ 

### Problem 1.4

In [8]:
import numpy as np

n, d = 20, 30
x = np.random.randn(n, d)

#def check_rbf_kernel(x, sigma=1.):
    
    
#check_rbd_kernel(x)

## Part 2: The Representer Theorem 

In [1]:
from urllib.request import urlretrieve
from os import path as osp

def retrieve(file_name, semester='fall22', course='371', homework=8):
    if osp.exists(file_name):
        print('Using previously downloaded file {}'.format(file_name))
    else:
        fmt = 'https://www2.cs.duke.edu/courses/{}/compsci{}/homework/{}/{}'
        url = fmt.format(semester, course, homework, file_name)
        urlretrieve(url, file_name)
        print('Downloaded file {}'.format(file_name))

In [5]:
import pickle
ad_data_file = 'ad.pickle'
retrieve(ad_data_file, homework=6)
with open(ad_data_file, 'rb') as file:
    ad_data = pickle.load(file)

Using previously downloaded file ad.pickle


### Problem 2.1 (Exam Style)

The representer theorem does hold for $L_{\text{reg}}(v) = \left\lVert v \right\rVert^2 + CL_T(v)$ because it matches the general formulation of the representer theorem (assuming $v$ is a training observation where $v \in \mathbb{R}^N$). The representer theorem requires a strictly increasing function $R$ from $\mathbb{R}_+$ to $\mathbb{R}$ and any function $S$ from $\mathbb{R}^N$ to $\mathbb{R}$.

We see that for the function $R(a) = a^2$ and the function $S(v) = CL_T(v)$, the LRC risk function can be rewritten as $L_{\text{reg}}(v) = R(\left\lVert v \right\rVert) + S(v)$, which matches the general formulation of the representer theorem. Therefore, the reprenter theorem holds for LRCs trained with regularization.

### Problem 2.2 (Exam Style)

The representer theorem does not hold for $L_T(v) = -\frac{1}{N}\sum\limits_{n=1}^N[y_n\log p_n + (1-y_n)\log(1-p_n)]$ where $p_n = \frac{1}{e^{-w^Tx_n-b}}$ because we violate the proof assumption that we have a strictly increasing function $R$ from $\mathbb{R}_+$ to $\mathbb{R}$ in our training risk function. All we have in the standard cross-entropy loss function is a single term, which serves as the function $S$ from $\mathbb{R}^N$ to $R$.

The proof is violated specifically at inequality (6) in the class notes, where we use the function $R$ to justify the implication that $R(\left\lVert w \right\rVert) < R(\left\lVert w* \right\rVert)$ (where $w = w^* - u, u \neq 0$). 

Since the proof also shows that our function $S$ has the property $S(w^Tx_1+b,...,w^Tx_N+b) = S((w^*)^Tx_1+b,...,(w^*)^Tx_N+b)$, the proof relies on the fact that inequality (6) exists to prove that $L(w,b) < L(w^*,b)$. Since this statement isn't valid anymore, we can't say that $w^*$ is the optimal vector, and therefore the vector $u \in X^\perp$ can be nonzero, meaning the hyerplane cannot be expressed as a linear combination of the input vectors $x_1,...,x_N \in X$.

### Problem 2.3

In [14]:
def evaluate(h, data, h_name):
    def accuracy(s):
        sx, sy = s['x'], s['y']
        return h.score(sx, sy) * 100

    train, test = data['train'], data['test']
    f = '{:s}:\n\ttraining accuracy is {:.2f} percent,' +\
        '\n\ttest accuracy is {:.2f} percent,'
    print(f.format(h_name, accuracy(train), accuracy(test)))

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def experiment(h, data, h_name): 
    # cross-validates classifier and fits to determine a regularization constant
    cs = np.logspace(-3,2,15)
    h = GridSearchCV(h, {'C': cs})
    h = h.fit(data['train']['x'], data['train']['y'])
    h = h.best_estimator_
    
    # obtain training and testing accuracy
    evaluate(h, data, h_name)

    #printing remaining required information
    print('\tthe best value for the regularization constant C is {},'.format(h.C))
    print('\tthere are {} training samples of dimensionality {},'.format(len(data['train']['x']), len(data['train']['x'][0])))
    print('\tthe number of linearly independent training samples is {}'.format(np.linalg.matrix_rank(data['train']['x'])))
    
    # calculating residuals for optimal and random vectors (only applicable to linear classifiers)
    if hasattr(h, 'coef_'):
        w = h.coef_.flatten()
        random_w = np.random.rand(len(w))
        
        print('\tthe residual for the optimal vector found by the training algorithm is %.'.format(residual(w, data['train']['x'])))
        print('\tthe residual for a random vector is %.'.format(residual(random_w, data['train']['x'])))
        return h, w
    else:
        return h
    
# helper function to calculate residual
def residual(vector, data):
    # computing coefficients from the data points
    beta = np.linalg.lstsq(a=np.transpose(data), b=vector)[0]

    sum_array = np.empty(shape=(np.size(data[0]),1))
    
    print(np.size(data))
    for d in range(np.size(data)-1):
        np.append(sum_array, beta[d]*data[d])
        
    return np.linalg.norm(vector - sum_array)
    
experiment(LogisticRegression(max_iter=1000), ad_data, 'LRC');

LRC:
	training accuracy is 99.15 percent,
	test accuracy is 96.78 percent,
	the best value for the regularization constant C is 0.0610540229658533,
	there are 1179 training samples of dimensionality 1430,
	the number of linearly independent training samples is 561


/var/folders/kl/s_1z5b7d0hd63whkbcpmn10w0000gn/T/ipykernel_72569/2949515679.py:33: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta = np.linalg.lstsq(a=np.transpose(data), b=vector)[0]


1685970


IndexError: index 1179 is out of bounds for axis 0 with size 1179

Such high dimesionality - low probability of picking a vector at rnadom and having a residual close to zero

### Problem 2.4

In [ ]:
from sklearn.svm import SVC

svm, w_svm = experiment(SVC(kernel='linear'), ad_data, 'SVM')

## Part 3: Linear and Nonlinear SVMs 

In [3]:
data_2d_file_name = 'data.pickle'
retrieve(data_2d_file_name)
with open(data_2d_file_name, 'rb') as file:
    data_2d = pickle.load(file)

Using previously downloaded file data.pickle


In [4]:
show_file = 'show.py'
retrieve(show_file)
from show import show_classification

Using previously downloaded file show.py


### Problem 3.1 (Exam Style except for Running the Code)

In [ ]:
experiment(SVC(kernel='rbf'), ad_data, 'RBF SVM')

### Problem 3.2 (Exam Style except for Running the Code)

In [ ]:
linear_svm, _ = experiment(SVC(kernel='linear'), data_2d, 'linear SVM')
show_classification(linear_svm, data_2d['train'], 'training', 'linear SVM')

In [ ]:
show_classification(lienar_svm, data_2d['test'], 'test', 'linear SVM')

### Problem 3.3 (Exam Style except for Running the Code)

In [ ]:
rbf_svm = experiment(SVC(kernel='rbf'), data_2d, 'RBF SVM')
show_classification(rbf_svm, data_2d['train'], 'training', 'RBF SVM')

In [ ]:
show_classification(rbf_svm, data_2d['train'], 'test', 'RBF SVM')